In [1]:
import os
import pandas as pd
import numpy as np
import faiss
from openai import OpenAI
from dotenv import load_dotenv
from tqdm import tqdm

# .env에 저장된 OPENAI_API_KEY 사용
load_dotenv()
client = OpenAI()

In [3]:
# 데이터 불러오기
df_full = pd.read_csv('rag_chunks_with_embeddings.csv', encoding='utf-8-sig', engine='python')

# 데이터 1% 랜덤 샘플링 (random_state로 고정하면 재현 가능)
df_sampled = df_full.sample(frac=0.01, random_state=42).reset_index(drop=True)

# OpenAI 임베딩 함수는 그대로 사용
def embed_text(text, model="text-embedding-ada-002"):
    response = client.embeddings.create(
        input=[text],
        model=model
    )
    return response.data[0].embedding

# 샘플 데이터에 대해 임베딩 생성
from tqdm import tqdm
new_embeddings = []
for idx, text in tqdm(enumerate(df_sampled['text']), total=len(df_sampled)):
    try:
        embedding = embed_text(text)
        new_embeddings.append(embedding)
    except Exception as e:
        print(f"{idx}번 임베딩 오류: {e}")
        new_embeddings.append([0.0] * 1536)

df_sampled['embedding'] = new_embeddings
df_sampled.to_csv('rag_chunks_with_embeddings_sampled.csv', index=False, encoding='utf-8-sig')


100%|██████████| 1234/1234 [08:18<00:00,  2.48it/s]


In [5]:
df = pd.read_csv('rag_chunks_with_embeddings_sampled.csv', encoding='utf-8-sig', engine='python')

df['embedding'] = df['embedding'].apply(eval)  # 문자열을 리스트로 변환
embeddings = np.array(df['embedding'].tolist()).astype('float32')

In [6]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)

faiss.normalize_L2(embeddings)  # 정규화
index.add(embeddings)

In [7]:
def llm_api_call(prompt, model="gpt-4o-mini", temperature=0.7, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

In [8]:
def search_similar_chunks(query, top_k=5):
    query_vec = embed_text(query)
    query_vec = np.array(query_vec).astype('float32').reshape(1, -1)
    faiss.normalize_L2(query_vec)

    D, I = index.search(query_vec, top_k)
    return df.iloc[I[0]]

In [9]:
def query_vitamin_chatbot(user_question, top_k=5):
    retrieved_chunks = search_similar_chunks(user_question, top_k=top_k)
    context = "\n".join(retrieved_chunks['text'].tolist())

    prompt = f"""다음은 건강기능식품 비타민 관련 정보입니다.
다음을 참고하여 사용자 질문에 성의껏 답변해주세요.

{context}

사용자 질문: {user_question}

답변:"""

    return llm_api_call(prompt)

In [12]:
question = "위장 장애를 완화할 수 있는 비타민 추천"
answer = query_vitamin_chatbot(question)
print(answer)

위장 장애를 완화하는 데 도움을 줄 수 있는 특정 비타민은 없지만, 일반적으로 소화 건강에 긍정적인 영향을 미칠 수 있는 비타민과 미네랄이 있습니다. 예를 들어, 비타민B군은 에너지 대사와 소화에 중요한 역할을 하며, 특히 비타민B1, 비타민B2, 비타민B6, 비타민B12 등이 소화 기능에 도움을 줄 수 있습니다. 

또한, 비타민C는 면역 체계를 지원하고 결합조직 형성에 필요합니다. 이러한 비타민을 포함한 건강기능식품으로는 '피로개선 활력비타민'과 같은 제품이 있습니다. 이 제품에는 여러 비타민B군이 포함되어 있어 에너지 대사와 소화 건강에 도움을 줄 수 있습니다.

그러나 위장 장애가 지속되거나 심각한 경우에는 전문가와 상담하여 정확한 원인을 파악하고 적절한 치료를 받는 것이 중요합니다.
